In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!mamba install -q -c conda-forge rdkit
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-spline-conv -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric

✨🍰✨ Everything looks OK!
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  err

In [ ]:


import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import numpy as np


# Additional imports that might be useful based on the original script
from typing import List, Tuple, Union
from argparse import Namespace
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from typing import List, Tuple
from argparse import Namespace

import torch
from torch.utils.data import Dataset, DataLoader
from typing import List, Tuple


In [ ]:
# After the imports, add these constants:

# Atom feature sizes
MAX_ATOMIC_NUM = 100
ATOM_FEATURES = {
    'atomic_num': list(range(MAX_ATOMIC_NUM)),
    'degree': [0, 1, 2, 3, 4, 5],
    'formal_charge': [-1, -2, 1, 2, 0],
    'chiral_tag': [0, 1, 2, 3],
    'num_Hs': [0, 1, 2, 3, 4],
    'hybridization': [
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3,
        Chem.rdchem.HybridizationType.SP3D,
        Chem.rdchem.HybridizationType.SP3D2
    ],
}

# len(choices) + 1 to include room for uncommon values; + 2 at end for IsAromatic and mass
ATOM_FDIM = sum(len(choices) + 1 for choices in ATOM_FEATURES.values()) + 2
BOND_FDIM = 14

In [ ]:
def onek_encoding_unk(value: int, choices: List[int]) -> List[int]:
    """
    Creates a one-hot encoding.
    """
    # Initialize a list of zeros with length equal to the number of choices plus one
    encoding = [0] * (len(choices) + 1)

    # Find the index of the value in the list of choices
    # If the value is not in choices, set index to -1 (for the "unknown" category)
    index = choices.index(value) if value in choices else -1

    # Set the corresponding position in the encoding list to 1
    # If the value was found in choices, encoding[index] is set to 1
    # If the value was not found, encoding[-1] is set to 1 (the "unknown" category)
    encoding[index] = 1

    # Return the one-hot encoded list
    return encoding


In [ ]:
def atom_features(atom: Chem.rdchem.Atom, functional_groups: List[int] = None) -> List[Union[bool, int, float]]:
    """
    Builds a feature vector for an atom.
    """
    features = onek_encoding_unk(atom.GetAtomicNum() - 1, ATOM_FEATURES['atomic_num']) + \
           onek_encoding_unk(atom.GetTotalDegree(), ATOM_FEATURES['degree']) + \
           onek_encoding_unk(atom.GetFormalCharge(), ATOM_FEATURES['formal_charge']) + \
           onek_encoding_unk(int(atom.GetChiralTag()), ATOM_FEATURES['chiral_tag']) + \
           onek_encoding_unk(int(atom.GetTotalNumHs()), ATOM_FEATURES['num_Hs']) + \
           onek_encoding_unk(int(atom.GetHybridization()), ATOM_FEATURES['hybridization']) + \
           [1 if atom.GetIsAromatic() else 0] + \
           [atom.GetMass() * 0.01]  # scaled to about the same range as other features
    if functional_groups is not None:
        features += functional_groups
    return features

In [ ]:
def bond_features(bond: Chem.rdchem.Bond) -> List[Union[bool, int, float]]:
    """
    Builds a feature vector for a bond.
    """
    if bond is None:
        fbond = [1] + [0] * (BOND_FDIM - 1)
    else:
        bt = bond.GetBondType()
        fbond = [
            0,  # bond is not None
            bt == Chem.rdchem.BondType.SINGLE,
            bt == Chem.rdchem.BondType.DOUBLE,
            bt == Chem.rdchem.BondType.TRIPLE,
            bt == Chem.rdchem.BondType.AROMATIC,
            (bond.GetIsConjugated() if bt is not None else 0),
            (bond.IsInRing() if bt is not None else 0)
        ]
        fbond += onek_encoding_unk(int(bond.GetStereo()), list(range(6)))
    return fbond

In [ ]:
class MolGraph:
    def __init__(self, smiles: str, args: Namespace):
        self.smiles = smiles
        self.n_atoms = 0
        self.n_bonds = 0
        self.f_atoms = []
        self.f_bonds = []
        self.a2b = []
        self.b2a = []
        self.b2revb = []
        self.bonds = []

        # Convert smiles to molecule
        mol = Chem.MolFromSmiles(smiles)
        self.n_atoms = mol.GetNumAtoms()

        # Get atom features
        for atom in mol.GetAtoms():
            self.f_atoms.append(atom_features(atom))

        # Initialize a2b for each atom
        self.a2b = [[] for _ in range(self.n_atoms)]

        # Get bond features
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            f_bond = bond_features(bond)

            # Add bonds in both directions
            self.f_bonds.append(f_bond)
            self.f_bonds.append(f_bond)

            # Update index mappings
            b1 = self.n_bonds
            b2 = b1 + 1
            self.a2b[i].append(b1)
            self.a2b[j].append(b2)
            self.b2a.extend([i, j])
            self.b2revb.extend([b2, b1])
            self.n_bonds += 2
            self.bonds.append([i, j])

        # Pad a2b lists
        max_bonds = max(len(in_bonds) for in_bonds in self.a2b)
        self.a2b = [bonds + [0] * (max_bonds - len(bonds)) for bonds in self.a2b]



In [ ]:
class BatchMolGraph:
    def __init__(self, mol_graphs: List[MolGraph]):
        self.smiles_batch = [mg.smiles for mg in mol_graphs]
        self.n_mols = len(self.smiles_batch)

        self.atom_fdim = len(mol_graphs[0].f_atoms[0])
        self.bond_fdim = len(mol_graphs[0].f_bonds[0]) if mol_graphs[0].f_bonds else 0

        self.n_atoms = 0
        self.n_bonds = 0
        self.a_scope = []
        self.b_scope = []

        f_atoms = []
        f_bonds = []
        a2b = []
        b2a = []
        b2revb = []
        bonds = []

        for mol_graph in mol_graphs:
            f_atoms.extend(mol_graph.f_atoms)
            f_bonds.extend(mol_graph.f_bonds)

            for a in range(mol_graph.n_atoms):
                a2b.append([b + self.n_bonds for b in mol_graph.a2b[a]])

            for b in range(mol_graph.n_bonds):
                b2a.append(self.n_atoms + mol_graph.b2a[b])
                b2revb.append(self.n_bonds + mol_graph.b2revb[b])

            for bond in mol_graph.bonds:
                bonds.append([b + self.n_atoms for b in bond])

            self.a_scope.append((self.n_atoms, mol_graph.n_atoms))
            self.b_scope.append((self.n_bonds, mol_graph.n_bonds))
            self.n_atoms += mol_graph.n_atoms
            self.n_bonds += mol_graph.n_bonds

        self.max_num_bonds = max(1, max(len(in_bonds) for in_bonds in a2b))

        self.f_atoms = torch.FloatTensor(f_atoms)
        self.f_bonds = torch.FloatTensor(f_bonds)
        self.a2b = torch.LongTensor([a2b[a] + [0] * (self.max_num_bonds - len(a2b[a])) for a in range(self.n_atoms)])
        self.b2a = torch.LongTensor(b2a)
        self.b2revb = torch.LongTensor(b2revb)
        self.bonds = torch.LongTensor(bonds)

    def get_components(self) -> Tuple[torch.FloatTensor, torch.FloatTensor,
                                      torch.LongTensor, torch.LongTensor, torch.LongTensor,
                                      List[Tuple[int, int]], List[Tuple[int, int]], torch.LongTensor]:
        return self.f_atoms, self.f_bonds, self.a2b, self.b2a, self.b2revb, self.a_scope, self.b_scope, self.bonds

In [ ]:
def mol2graph(smiles_batch: List[str], args: Namespace) -> BatchMolGraph:
    mol_graphs = []
    for smiles in smiles_batch:
        mol_graph = MolGraph(smiles, args)
        mol_graphs.append(mol_graph)

    return BatchMolGraph(mol_graphs)

In [ ]:
def extract_features_from_mol_graph(mol_graph: MolGraph) -> dict:
    features = {
        'f_atoms': torch.FloatTensor(mol_graph.f_atoms),
        'f_bonds': torch.FloatTensor(mol_graph.f_bonds),
    }

    # Handle potentially problematic attributes
    try:
        features['a2b'] = torch.LongTensor(mol_graph.a2b)
    except ValueError:
        print(f"Warning: a2b conversion failed for molecule {mol_graph.smiles}")
        features['a2b'] = torch.LongTensor([])

    try:
        features['b2a'] = torch.LongTensor(mol_graph.b2a)
    except ValueError:
        print(f"Warning: b2a conversion failed for molecule {mol_graph.smiles}")
        features['b2a'] = torch.LongTensor([])

    try:
        features['b2revb'] = torch.LongTensor(mol_graph.b2revb)
    except ValueError:
        print(f"Warning: b2revb conversion failed for molecule {mol_graph.smiles}")
        features['b2revb'] = torch.LongTensor([])

    try:
        features['bonds'] = torch.LongTensor(np.array(mol_graph.bonds).transpose())
    except ValueError:
        print(f"Warning: bonds conversion failed for molecule {mol_graph.smiles}")
        features['bonds'] = torch.LongTensor([])

    return features

In [ ]:
# # Read the Excel file
# df = pd.read_excel('/content/GWP_PUBCHEM_with_smiles_edited.xlsx')

# # Get the first 20 SMILES strings (or however many you want to test)
# smiles_list = df['smiles'].head(5).tolist()

# # Create a simple Namespace object for args
# args = Namespace(atom_messages=False)

# # Convert SMILES to BatchMolGraph
# batch_graph = mol2graph(smiles_list, args)

# # Print batch information
# print(f"Number of molecules: {batch_graph.n_mols}")
# print(f"Total number of atoms: {batch_graph.n_atoms}")
# print(f"Total number of bonds: {batch_graph.n_bonds}")
# print(f"Atom feature dimension: {batch_graph.atom_fdim}")
# print(f"Bond feature dimension: {batch_graph.bond_fdim}")
# print(f"Max number of bonds per atom: {batch_graph.max_num_bonds}")
# print("\n")

# # Extract and print features for each molecule in the batch
# f_atoms, f_bonds, a2b, b2a, b2revb, a_scope, b_scope = batch_graph.get_components()

# for i in range(batch_graph.n_mols):
#     a_start, a_size = a_scope[i]
#     b_start, b_size = b_scope[i]

#     print(f"Molecule {i+1}: {df['Name'].iloc[i]}")
#     print("SMILES:", smiles_list[i])
#     print(f"f_atoms: {f_atoms[a_start:a_start+a_size].shape}")
#     print(f"f_bonds: {f_bonds[b_start:b_start+b_size].shape}")
#     print(f"a2b: {a2b[a_start:a_start+a_size].shape}")
#     print(f"b2a: {b2a[b_start:b_start+b_size].shape}")
#     print(f"b2revb: {b2revb[b_start:b_start+b_size].shape}")
#     print("f_atoms (first atom):", f_atoms[a_start])
#     print("f_bonds (first bond):", f_bonds[b_start] if b_size > 0 else "No bonds")
#     print("\n")

In [ ]:
def get_activation_function(activation: str) -> nn.Module:
    if activation == 'ReLU':
        return nn.ReLU()
    elif activation == 'LeakyReLU':
        return nn.LeakyReLU(0.1)
    elif activation == 'PReLU':
        return nn.PReLU()
    elif activation == 'tanh':
        return nn.Tanh()
    elif activation == 'SELU':
        return nn.SELU()
    elif activation == 'ELU':
        return nn.ELU()
    else:
        raise ValueError(f'Activation "{activation}" not supported.')

In [ ]:
def index_select_ND(source: torch.Tensor, index: torch.Tensor) -> torch.Tensor:
    index_size = index.size()
    suffix_dim = source.size()[1:]
    final_size = index_size + suffix_dim
    target = source.index_select(dim=0, index=index.view(-1))
    target = target.view(final_size)
    target[index==0] = 0
    return target

In [ ]:
class MPNEncoder(nn.Module):
    def __init__(self, atom_fdim, bond_fdim, hidden_size, bias, depth, dropout, activation, device):
        super(MPNEncoder, self).__init__()
        self.atom_fdim = atom_fdim
        self.bond_fdim = bond_fdim
        self.hidden_size = hidden_size
        self.bias = bias
        self.depth = depth
        self.dropout = dropout
        self.device = device
        self.layers_per_message = 1

        self.dropout_layer = nn.Dropout(p=self.dropout).to(device)
        self.act_func = get_activation_function(activation).to(device)
        self.W_i_atom = nn.Linear(self.atom_fdim, self.hidden_size, bias=self.bias).to(device)
        self.W_i_bond = nn.Linear(self.bond_fdim, self.hidden_size, bias=self.bias).to(device)

        w_h_input_size_atom = self.hidden_size + self.bond_fdim
        self.W_h_atom = nn.Linear(w_h_input_size_atom, self.hidden_size, bias=self.bias).to(device)

        w_h_input_size_bond = self.hidden_size

        for depth in range(self.depth - 1):
            self._modules[f'W_h_{depth}'] = nn.Linear(w_h_input_size_bond, self.hidden_size, bias=self.bias).to(device)

        self.W_o = nn.Linear((self.hidden_size) * 2, self.hidden_size).to(device)
        self.gru = BatchGRU(self.hidden_size).to(device)
        self.lr = nn.Linear(self.hidden_size * 3, self.hidden_size, bias=self.bias).to(device)

    def forward(self, mol_graph: BatchMolGraph, features_batch=None, batch_mask=None) -> torch.FloatTensor:
        f_atoms, f_bonds, a2b, b2a, b2revb, a_scope, b_scope, bonds = mol_graph.get_components()
        f_atoms, f_bonds, a2b, b2a, b2revb = (f_atoms.to(self.device), f_bonds.to(self.device), a2b.to(self.device),
                                              b2a.to(self.device), b2revb.to(self.device))
        input_atom = self.W_i_atom(f_atoms)
        input_atom = self.act_func(input_atom)
        message_atom = input_atom.clone()

        input_bond = self.W_i_bond(f_bonds)
        message_bond = self.act_func(input_bond)
        input_bond = self.act_func(input_bond)

        for depth in range(self.depth - 1):
            agg_message = index_select_ND(message_bond, a2b)
            agg_message = agg_message.sum(dim=1) * agg_message.max(dim=1)[0]
            message_atom = message_atom + agg_message

            rev_message = message_bond[b2revb]
            message_bond = message_atom[b2a] - rev_message

            message_bond = self._modules[f'W_h_{depth}'](message_bond)
            message_bond = self.dropout_layer(self.act_func(input_bond + message_bond))

        agg_message = index_select_ND(message_bond, a2b)
        agg_message = agg_message.sum(dim=1) * agg_message.max(dim=1)[0]
        agg_message = self.lr(torch.cat([agg_message, message_atom, input_atom], 1))
        agg_message = self.gru(agg_message, a_scope)

        atom_hiddens = self.act_func(self.W_o(agg_message))
        atom_hiddens = self.dropout_layer(atom_hiddens)

        print("MPNEncoder output shape:", atom_hiddens[1:].shape)

        return atom_hiddens[1:]

class BatchGRU(nn.Module):
    def __init__(self, hidden_size=300):
        super(BatchGRU, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, batch_first=True, bidirectional=True)
        self.bias = nn.Parameter(torch.Tensor(self.hidden_size))
        self.bias.data.uniform_(-1.0 / math.sqrt(self.hidden_size), 1.0 / math.sqrt(self.hidden_size))

    def forward(self, node, a_scope):
        hidden = node
        message = F.relu(node + self.bias)
        MAX_atom_len = max([a_size for a_start, a_size in a_scope])
        message_lst = []
        hidden_lst = []
        for i, (a_start, a_size) in enumerate(a_scope):
            if a_size == 0:
                assert 0
            cur_message = message.narrow(0, a_start, a_size)
            cur_hidden = hidden.narrow(0, a_start, a_size)
            hidden_lst.append(cur_hidden.max(0)[0].unsqueeze(0).unsqueeze(0))
            cur_message = torch.nn.ZeroPad2d((0, 0, 0, MAX_atom_len - cur_message.shape[0]))(cur_message)
            message_lst.append(cur_message.unsqueeze(0))

        message_lst = torch.cat(message_lst, 0)
        hidden_lst = torch.cat(hidden_lst, 1)
        hidden_lst = hidden_lst.repeat(2, 1, 1)
        cur_message, cur_hidden = self.gru(message_lst, hidden_lst)

        cur_message_unpadding = []
        for i, (a_start, a_size) in enumerate(a_scope):
            cur_message_unpadding.append(cur_message[i, :a_size].view(-1, 2 * self.hidden_size))
        cur_message_unpadding = torch.cat(cur_message_unpadding, 0)

        message = torch.cat([torch.cat([message.narrow(0, 0, 1), message.narrow(0, 0, 1)], 1),
                             cur_message_unpadding], 0)
        return message

In [ ]:
class MoleculeDataset(Dataset):
    def __init__(self, smiles_list: List[str], labels: List[float], args):
        self.smiles_list = smiles_list
        self.labels = labels
        self.args = args

    def __len__(self):
        return len(self.smiles_list)

    def __getitem__(self, idx):
        smiles = self.smiles_list[idx]
        label = self.labels[idx]
        mol_graph = MolGraph(smiles, self.args)  # Create MolGraph instead of BatchMolGraph
        return mol_graph, label

def create_data_loader(smiles_list, labels, args, batch_size, shuffle=True):
    dataset = MoleculeDataset(smiles_list, labels, args)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=lambda x: x)

In [ ]:
class GNN(nn.Module):
    def __init__(self, args):
        super(GNN, self).__init__()
        self.encoder = MPNEncoder(
            atom_fdim=args.atom_fdim,
            bond_fdim=args.bond_fdim,
            hidden_size=args.hidden_size,
            bias=args.bias,
            depth=args.depth,
            dropout=args.dropout,
            activation=args.activation,
            device=args.device
        )
        self.ffn = nn.Linear(args.hidden_size, 1)

    def forward(self, batch):
        mol_graphs, _ = zip(*batch)
        batch_mol_graph = BatchMolGraph(mol_graphs)
        atom_hiddens = self.encoder(batch_mol_graph)

        # Process each molecule separately
        mol_vecs = []
        start = 0
        for _, a_size in batch_mol_graph.a_scope:
            if a_size == 0:
                mol_vecs.append(torch.zeros(self.encoder.hidden_size, device=atom_hiddens.device))
            else:
                mol_vecs.append(atom_hiddens[start:start+a_size].sum(dim=0))
            start += a_size
        mol_vecs = torch.stack(mol_vecs, dim=0)

        return self.ffn(mol_vecs).squeeze(-1)

In [ ]:
def compute_r2(y_true, y_pred):
    return r2_score(y_true, y_pred)

In [ ]:
def train(model, train_loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0

    for batch in train_loader:
        mol_graphs, labels = zip(*batch)
        labels = torch.FloatTensor(labels).to(device)

        optimizer.zero_grad()
        outputs = model(batch)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

def evaluate(model, val_loader, loss_fn, device):
    model.eval()
    total_loss = 0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            mol_graphs, labels = zip(*batch)
            labels = torch.FloatTensor(labels).to(device)

            outputs = model(batch)
            loss = loss_fn(outputs, labels)
            total_loss += loss.item()

            all_predictions.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print(f"Number of predictions: {len(all_predictions)}")
    print(f"Number of labels: {len(all_labels)}")

    return total_loss / len(val_loader), np.array(all_predictions), np.array(all_labels)

In [ ]:
import pandas as pd
from argparse import Namespace
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from sklearn.metrics import r2_score
# Load your data
df = pd.read_excel('/content/GWP_PUBCHEM_with_smiles_edited.xlsx')
smiles_list = df['smiles'].tolist()
labels = df['GWP Hierarchist (kg CO2eq/ kg) '].tolist()

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the labels and transform them
labels_normalized = scaler.fit_transform(np.array(labels).reshape(-1, 1)).flatten()

# Split data into train and validation sets
train_smiles, val_smiles, train_labels, val_labels = train_test_split(smiles_list, labels_normalized, test_size=0.2, random_state=42)

# Set up arguments
args = Namespace(
    atom_fdim=133,
    bond_fdim=14,
    hidden_size=64,
    bias=False,
    depth=3,
    dropout=0.2,
    activation='ReLU',
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

# Create data loaders
train_loader = create_data_loader(train_smiles, train_labels, args, batch_size=128)
val_loader = create_data_loader(val_smiles, val_labels, args, batch_size=128, shuffle=False)

# Initialize model, optimizer, and loss function
model = GNN(args).to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Training loop
num_epochs = 170
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, loss_fn, args.device)
    val_loss, val_predictions, val_labels = evaluate(model, val_loader, loss_fn, args.device)
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Calculate R2 score
val_r2 = r2_score(val_labels, val_predictions)
print(f"Validation R2 Score: {val_r2:.4f}")

# If you want to get the R2 score on the original scale
val_predictions_original = scaler.inverse_transform(val_predictions.reshape(-1, 1)).flatten()
val_labels_original = scaler.inverse_transform(val_labels.reshape(-1, 1)).flatten()
val_r2_original = r2_score(val_labels_original, val_predictions_original)
print(f"Validation R2 Score (original scale): {val_r2_original:.4f}")

# Save the model
torch.save(model.state_dict(), 'gnn_model.pth')

MPNEncoder output shape: torch.Size([1352, 64])
MPNEncoder output shape: torch.Size([385, 64])
MPNEncoder output shape: torch.Size([407, 64])
Number of predictions: 41
Number of labels: 41
Epoch 1/170, Train Loss: 1.6256, Val Loss: 0.5177
MPNEncoder output shape: torch.Size([1282, 64])
MPNEncoder output shape: torch.Size([455, 64])
MPNEncoder output shape: torch.Size([407, 64])
Number of predictions: 41
Number of labels: 41
Epoch 2/170, Train Loss: 1.3544, Val Loss: 0.5601
MPNEncoder output shape: torch.Size([1360, 64])
MPNEncoder output shape: torch.Size([377, 64])
MPNEncoder output shape: torch.Size([407, 64])
Number of predictions: 41
Number of labels: 41
Epoch 3/170, Train Loss: 1.1314, Val Loss: 0.4137
MPNEncoder output shape: torch.Size([1374, 64])
MPNEncoder output shape: torch.Size([363, 64])
MPNEncoder output shape: torch.Size([407, 64])
Number of predictions: 41
Number of labels: 41
Epoch 4/170, Train Loss: 1.0123, Val Loss: 0.4777
MPNEncoder output shape: torch.Size([1350, 6